# Scraping Union List of Sanborn Maps

This jupyter notebook will have the code to scrap the list of sandborn maps from the Berkley library website and put them into a CSV format. 

In [1]:
#Importing libraries
import requests #Lets us make requests to a website
from bs4 import BeautifulSoup #Importing Beautiful Soup for webscraping

In [2]:
# saving a website url
union_list = "http://www.lib.berkeley.edu/EART/sanborn_union_list"

r = requests.get(union_list)
soup=BeautifulSoup(r.text, 'html.parser')

#Finding all the links on the main page
all_links = []
for link in soup.findAll('a'):
    all_links.append(link.get('href'))
print('All links: %s', len(all_links))


All links: %s 72


In [123]:
state_links = []
for link in all_links:
    if link is not None:
        final_url_element = link.split('/')[-1]
    if final_url_element.startswith('sanbul'):
        state_links.append(link)
        
state_links = state_links[0:54]

state_links

['http://www.lib.berkeley.edu/EART/sanbul_AL.html',
 'http://www.lib.berkeley.edu/EART/sanbul_AK.html',
 'http://www.lib.berkeley.edu/EART/sanbul_AZ.html',
 'http://www.lib.berkeley.edu/EART/sanbul_AR.html',
 'http://www.lib.berkeley.edu/EART/sanbul_CA_AB.html',
 'http://www.lib.berkeley.edu/EART/sanbul_CO.html',
 'http://www.lib.berkeley.edu/EART/sanbul_CT.html',
 'http://www.lib.berkeley.edu/EART/sanbul_DE.html',
 'http://www.lib.berkeley.edu/EART/sanbul_FL.html',
 'http://www.lib.berkeley.edu/EART/sanbul_GA.html',
 'http://www.lib.berkeley.edu/EART/sanbul_HI.html',
 'http://www.lib.berkeley.edu/EART/sanbul_ID.html',
 'http://www.lib.berkeley.edu/EART/sanbul_IL_A.html',
 'http://www.lib.berkeley.edu/EART/sanbul_IN.html',
 'http://www.lib.berkeley.edu/EART/sanbul_IA_AE.html',
 'http://www.lib.berkeley.edu/EART/sanbul_KS.html',
 'http://www.lib.berkeley.edu/EART/sanbul_KY_AG.html',
 'http://www.lib.berkeley.edu/EART/sanbul_LA.html',
 'http://www.lib.berkeley.edu/EART/sanbul_ME_AL.html'

In [70]:
#Trying out code using a single

#Importing libraries
import csv #Lets us write to a csv file
import requests #Lets us make requests to a website
from bs4 import BeautifulSoup #Importing Beautiful Soup for webscraping

#Trying out code for one url
state_list = "http://www.lib.berkeley.edu/EART/sanbul_AK.html"
r = requests.get(state_list)
soup_state = BeautifulSoup(r.text, 'html.parser')
    
#Getting the data in a more workable format
html = list(soup_state.children)[2]
body = list(html.children)[1]
core = list(body.children)

string_soup = []
for item in core:
    string= str(item)
    string_soup.append(string)
    
cities = []
dates = []
libs = []
city = ""
date = ""
lib= ""

for item in string_soup:
    if "city" in item:
        city = item
    elif "date" in item:
        date = item
    elif "class=\"lib\"" in item:
        lib =item
        cities.append(city[16:-4])
        dates.append(date[18:-6])
        libs.append(lib[17:-6])


with open('results.csv','w') as outfile:
    rowlists = zip(cities, dates, libs)
    writer = csv.writer(outfile)
    for row in rowlists:
        writer.writerows([row])

In [124]:
#Trying to use a loop

#Importing libraries
import csv #Lets us write to a csv file
import requests #Lets us make requests to a website
from bs4 import BeautifulSoup #Importing Beautiful Soup for webscraping

#Creating placeholder lists and variable to use in the loop
cities = []
dates = []
libs = []
city = ""
date = ""
lib= ""
states = []

for state in state_links:
    #parsing the url
    if state is not None:
        state_list = state
        r = requests.get(state_list)
        soup_state = BeautifulSoup(r.text, 'html.parser')

        #Getting the data in a more workable format
        html = list(soup_state.children)[2]
        body = list(html.children)[1]
        core = list(body.children)
        
        #Transforming the data to strings - makes next part easier
        string_soup = []
        for item in core:
            string= str(item)
            string_soup.append(string)
            
        #Creating lists of each value
        for item in string_soup:
            if "city" in item:
                city = item
            elif "date" in item:
                date = item
            elif "class=\"lib\"" in item:
                #Doing the appending in the lib level because there will sometimes be multiple libraries for a specific date/city
                lib =item
                cities.append(city[16:-4])
                dates.append(date[18:-6])
                libs.append(lib[17:-6])
                states.append(state)

#Writing out the results as a CSV file
with open('results.csv','w') as outfile:
    rowlists = zip(states, cities, dates, libs)
    writer = csv.writer(outfile)
    for row in rowlists:
        writer.writerows([row])

4290

4290

[]